In [9]:
from pathlib import Path

import numpy as np
import pandas as pd
import gensim
import jamo
from eunjeon import Mecab
from tqdm.auto import tqdm

tqdm.pandas()

c:\python37\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [10]:
DATA_TYPE = 'full'
MECAB_TYPE = 'nouns' # nouns or morphs

if DATA_TYPE.startswith('outer'):
    TEXT_COLUMNS = ['headline', 'lede']
elif DATA_TYPE.startswith('inner'):
    TEXT_COLUMNS = ['title', 'contents']
else:
    TEXT_COLUMNS = ['title', 'lede', 'contents']

In [11]:
PREPROCESSING_DIR = '2-preprocessing'
TOKENIZING_DIR = '3-tokenizing'
DATA_DIR = 'lol'

preprocessing_dir = Path(PREPROCESSING_DIR) / DATA_DIR
tokenizing_dir = Path(TOKENIZING_DIR) / DATA_DIR
tokenizing_dir.mkdir(parents=True, exist_ok=True)

In [12]:
mecab = Mecab()
if MECAB_TYPE == 'nouns':
    mecab_fn = lambda x: ' '.join(mecab.nouns(x)) if x is not np.nan else x
else:
    mecab_fn = lambda x: ' '.join(mecab.morphs(x)) if x is not np.nan else x

In [13]:
mecab_df = pd.read_csv(preprocessing_dir / ('news-%s.csv' % DATA_TYPE))
for column in TEXT_COLUMNS:
    mecab_df[column] = mecab_df[column].progress_apply(mecab_fn)
mecab_df.to_csv(tokenizing_dir / ('news-%s-%s.csv' % (DATA_TYPE, MECAB_TYPE)), index=False)
mecab_df

,date,date_input,date_modify,ranking,thumbnail,title,lede,office,contents,vod,view
0,20150801,20150801 12:49,20150801 12:49,1,1,핫 매치 리뷰 불량 학생 노동현 수능 대박 스토리,탈선 수능 만점 년 월 일 타이거즈 롤스터 정규 시즌 라운드 일 차 경기 팀 포스트 시즌,인벤,탈선 수능 만점 년 월 일 타이거즈 롤스터 정규 시즌 라운드 일 차 경기 팀 포스트...,0.0,198042
1,20150801,20150801 23:32,20150801 23:32,2,1,섬머 베누 소닉붐 뉴 클리어 인터뷰 앞 승리,금일 일 섬머 시즌 사상 베누 소닉붐 번 승리 기록 연패 번 승리 만큼 의미 감동,헝그리앱,금일 일 섬머 시즌 사상 베누 소닉붐 번 승리 기록 연패 번 승리 만큼 의미 감동 ...,0.0,8657
2,20150801,20150801 20:10,20150801 20:10,3,1,섬머 뱅 배준식 인터뷰 솔로 랭크 데 멘탈 중요,승리 소감 경기 오랜만 배성웅 벵 형 이지훈 이지훈 형 오랜만 때문 그 내,헝그리앱,승리 소감 경기 오랜만 배성웅 벵 형 이지훈 이지훈 형 오랜만 때문 그 내 편 다행...,0.0,416
3,20150801,20150801 23:31,20150801 23:31,4,1,롬 섬머 베누 소닉붐 꼴찌 반격 개시 동안 설움 설욕 대망 승리,롤 섬머 경기 세트 기세 스 베누 소닉붐 승리 때문 진에어 윙스 마지막 세트 플레이...,헝그리앱,롤 섬머 경기 세트 기세 스 베누 소닉붐 승리 때문 진에어 윙스 마지막 세트 플레이...,0.0,0
4,20150801,20150801 21:58,20150801 21:58,5,1,롬 섬머 베누 소닉붐 타 대승 기반 세트 승리,롤 섬머 경기 세트 시작 세트 때 진에어 윙스 스 베누 소닉붐 세트 때 미드 챔피언...,헝그리앱,롤 섬머 경기 세트 시작 세트 때 진에어 윙스 스 베누 소닉붐 세트 때 미드 챔피언...,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
51811,20200430,20200430 17:49,20200430 17:56,26,1,승강 전 다이브 승리 샌드박스 서라벌 세트 선승,일 종각 파크 섬머 스플릿 승강 전 최종 전 세트 샌드박스 게이밍 서라벌 게이밍 승...,인벤,일 종각 파크 섬머 스플릿 승강 전 최종 전 세트 샌드박스 게이밍 서라벌 게이밍 승...,0.0,1166
51812,20200430,20200430 16:14,20200430 16:31,27,1,승강 전 서라벌 샌드박스 베스트 라인업 맞대결,서머 최종 전 대결 서라벌 게이밍 위 샌드박스 게이밍 사진 라이엇 게임즈 제공 마지...,데일리e스포츠,마지막 승강 전 승리 서라벌 게이밍 샌드박스 게이밍 승강 전 기용 베스트 라인업 출...,0.0,1057
51813,20200430,20200430 17:50,20200430 17:50,28,1,일방 전투 승리 샌드박스 게이밍 서라벌 게이밍 상대 세트 승리,일 우리은행 승강 전 최종진 출전 샌드박스 게이밍 서라벌 게이밍 세트 경기 초반 기...,엑스포츠뉴스,일 우리은행 승강 전 최종진 출전 샌드박스 게이밍 서라벌 게이밍 세트 경기 초반 기...,0.0,835
51814,20200430,20200430 18:05,20200430 18:05,29,1,승강 전 샌드박스 카운터 기선 제압,고용준 기자 최후 승강 전 샌드박스 다이브 공세 흐름 샌드박스 승강 전 최종 전 서전,OSEN,최후 승강 전 샌드박스 다이브 공세 흐름 샌드박스 승강 전 최종 전 서전 승리 장식...,0.0,490


In [14]:
jamo_df = mecab_df.copy()
for column in TEXT_COLUMNS:
    jamo_df[column] = jamo_df[column].progress_apply(lambda x: jamo.j2hcj(jamo.h2j(x)) if x is not np.nan else x)
jamo_df.to_csv(tokenizing_dir / ('news-%s-%s-jamo.csv' % (DATA_TYPE, MECAB_TYPE)), index=False)
jamo_df

,date,date_input,date_modify,ranking,thumbnail,title,lede,office,contents,vod,view
0,20150801,20150801 12:49,20150801 12:49,1,1,ㅎㅏㅅ ㅁㅐㅊㅣ ㄹㅣㅂㅠ ㅂㅜㄹㄹㅑㅇ ㅎㅏㄱㅅㅐㅇ ㄴㅗㄷㅗㅇㅎㅕㄴ ㅅㅜㄴㅡㅇ ㄷㅐㅂ...,ㅌㅏㄹㅅㅓㄴ ㅅㅜㄴㅡㅇ ㅁㅏㄴㅈㅓㅁ ㄴㅕㄴ ㅇㅝㄹ ㅇㅣㄹ ㅌㅏㅇㅣㄱㅓㅈㅡ ㄹㅗㄹㅅㅡ...,인벤,ㅌㅏㄹㅅㅓㄴ ㅅㅜㄴㅡㅇ ㅁㅏㄴㅈㅓㅁ ㄴㅕㄴ ㅇㅝㄹ ㅇㅣㄹ ㅌㅏㅇㅣㄱㅓㅈㅡ ㄹㅗㄹㅅㅡ...,0.0,198042
1,20150801,20150801 23:32,20150801 23:32,2,1,ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㄴㅠ ㅋㅡㄹㄹㅣㅇㅓ ㅇㅣㄴㅌㅓㅂㅠ ㅇㅏㅍ ㅅㅡㅇㄹㅣ,ㄱㅡㅁㅇㅣㄹ ㅇㅣㄹ ㅅㅓㅁㅁㅓ ㅅㅣㅈㅡㄴ ㅅㅏㅅㅏㅇ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㅂㅓㄴ...,헝그리앱,ㄱㅡㅁㅇㅣㄹ ㅇㅣㄹ ㅅㅓㅁㅁㅓ ㅅㅣㅈㅡㄴ ㅅㅏㅅㅏㅇ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㅂㅓㄴ...,0.0,8657
2,20150801,20150801 20:10,20150801 20:10,3,1,ㅅㅓㅁㅁㅓ ㅂㅐㅇ ㅂㅐㅈㅜㄴㅅㅣㄱ ㅇㅣㄴㅌㅓㅂㅠ ㅅㅗㄹㄹㅗ ㄹㅐㅇㅋㅡ ㄷㅔ ㅁㅔㄴㅌ...,ㅅㅡㅇㄹㅣ ㅅㅗㄱㅏㅁ ㄱㅕㅇㄱㅣ ㅇㅗㄹㅐㄴㅁㅏㄴ ㅂㅐㅅㅓㅇㅇㅜㅇ ㅂㅔㅇ ㅎㅕㅇ ㅇㅣ...,헝그리앱,ㅅㅡㅇㄹㅣ ㅅㅗㄱㅏㅁ ㄱㅕㅇㄱㅣ ㅇㅗㄹㅐㄴㅁㅏㄴ ㅂㅐㅅㅓㅇㅇㅜㅇ ㅂㅔㅇ ㅎㅕㅇ ㅇㅣ...,0.0,416
3,20150801,20150801 23:31,20150801 23:31,4,1,ㄹㅗㅁ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㄲㅗㄹㅉㅣ ㅂㅏㄴㄱㅕㄱ ㄱㅐㅅㅣ ㄷㅗㅇㅇ...,ㄹㅗㄹ ㅅㅓㅁㅁㅓ ㄱㅕㅇㄱㅣ ㅅㅔㅌㅡ ㄱㅣㅅㅔ ㅅㅡ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㅅㅡㅇ...,헝그리앱,ㄹㅗㄹ ㅅㅓㅁㅁㅓ ㄱㅕㅇㄱㅣ ㅅㅔㅌㅡ ㄱㅣㅅㅔ ㅅㅡ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㅅㅡㅇ...,0.0,0
4,20150801,20150801 21:58,20150801 21:58,5,1,ㄹㅗㅁ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㅌㅏ ㄷㅐㅅㅡㅇ ㄱㅣㅂㅏㄴ ㅅㅔㅌㅡ ㅅㅡㅇㄹㅣ,ㄹㅗㄹ ㅅㅓㅁㅁㅓ ㄱㅕㅇㄱㅣ ㅅㅔㅌㅡ ㅅㅣㅈㅏㄱ ㅅㅔㅌㅡ ㄸㅐ ㅈㅣㄴㅇㅔㅇㅓ ㅇㅟㅇ...,헝그리앱,ㄹㅗㄹ ㅅㅓㅁㅁㅓ ㄱㅕㅇㄱㅣ ㅅㅔㅌㅡ ㅅㅣㅈㅏㄱ ㅅㅔㅌㅡ ㄸㅐ ㅈㅣㄴㅇㅔㅇㅓ ㅇㅟㅇ...,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
51811,20200430,20200430 17:49,20200430 17:56,26,1,ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ㄷㅏㅇㅣㅂㅡ ㅅㅡㅇㄹㅣ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅅㅓㄹㅏㅂㅓㄹ ㅅㅔㅌ...,ㅇㅣㄹ ㅈㅗㅇㄱㅏㄱ ㅍㅏㅋㅡ ㅅㅓㅁㅁㅓ ㅅㅡㅍㅡㄹㄹㅣㅅ ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ㅊㅚㅈㅗ...,인벤,ㅇㅣㄹ ㅈㅗㅇㄱㅏㄱ ㅍㅏㅋㅡ ㅅㅓㅁㅁㅓ ㅅㅡㅍㅡㄹㄹㅣㅅ ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ㅊㅚㅈㅗ...,0.0,1166
51812,20200430,20200430 16:14,20200430 16:31,27,1,ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ㅅㅓㄹㅏㅂㅓㄹ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅂㅔㅅㅡㅌㅡ ㄹㅏㅇㅣㄴㅇㅓㅂ ...,ㅅㅓㅁㅓ ㅊㅚㅈㅗㅇ ㅈㅓㄴ ㄷㅐㄱㅕㄹ ㅅㅓㄹㅏㅂㅓㄹ ㄱㅔㅇㅣㅁㅣㅇ ㅇㅟ ㅅㅐㄴㄷㅡㅂ...,데일리e스포츠,ㅁㅏㅈㅣㅁㅏㄱ ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ㅅㅡㅇㄹㅣ ㅅㅓㄹㅏㅂㅓㄹ ㄱㅔㅇㅣㅁㅣㅇ ㅅㅐㄴㄷㅡ...,0.0,1057
51813,20200430,20200430 17:50,20200430 17:50,28,1,ㅇㅣㄹㅂㅏㅇ ㅈㅓㄴㅌㅜ ㅅㅡㅇㄹㅣ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㄱㅔㅇㅣㅁㅣㅇ ㅅㅓㄹㅏㅂㅓㄹ ...,ㅇㅣㄹ ㅇㅜㄹㅣㅇㅡㄴㅎㅐㅇ ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ㅊㅚㅈㅗㅇㅈㅣㄴ ㅊㅜㄹㅈㅓㄴ ㅅㅐㄴㄷ...,엑스포츠뉴스,ㅇㅣㄹ ㅇㅜㄹㅣㅇㅡㄴㅎㅐㅇ ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ㅊㅚㅈㅗㅇㅈㅣㄴ ㅊㅜㄹㅈㅓㄴ ㅅㅐㄴㄷ...,0.0,835
51814,20200430,20200430 18:05,20200430 18:05,29,1,ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅋㅏㅇㅜㄴㅌㅓ ㄱㅣㅅㅓㄴ ㅈㅔㅇㅏㅂ,ㄱㅗㅇㅛㅇㅈㅜㄴ ㄱㅣㅈㅏ ㅊㅚㅎㅜ ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㄷㅏㅇㅣㅂ...,OSEN,ㅊㅚㅎㅜ ㅅㅡㅇㄱㅏㅇ ㅈㅓㄴ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㄷㅏㅇㅣㅂㅡ ㄱㅗㅇㅅㅔ ㅎㅡㄹㅡㅁ ...,0.0,490


In [7]:
clean_df = mecab_df.copy()
for column in TEXT_COLUMNS:
    clean_df[column] = clean_df[column].progress_apply(lambda x: ' '.join(gensim.utils.simple_preprocess(x)) if x is not np.nan else x)
clean_df.to_csv(tokenizing_dir / ('news-%s-%s-clean.csv' % (DATA_TYPE, MECAB_TYPE)), index=False)
clean_df

,date,date_input,date_modify,ranking,thumbnail,title,lede,office,contents,vod,view
0,20150801,20150801 12:49,20150801 12:49,1,1,챔스 매치 리뷰 불량 학생 노동현 수능 대박 스토리,탈선 어요 수능 만점 koo 타이거즈 kt 롤스터 정규 시즌 라운드 경기 펼쳐졌 모...,인벤,탈선 어요 수능 만점 koo 타이거즈 kt 롤스터 정규 시즌 라운드 경기 펼쳐졌 모...,0.0,198042
1,20150801,20150801 23:32,20150801 23:32,2,1,챔스 섬머 베누 소닉붐 클리어 사신 인터뷰 으로 승리 안겨 드리,금일 챔스 섬머 시즌 사상 베누 소닉붐 승리 기록 연패 마침내 승리 만큼 의미 감동,헝그리앱,금일 챔스 섬머 시즌 사상 베누 소닉붐 승리 기록 연패 마침내 승리 만큼 의미 감동...,0.0,8657
2,20150801,20150801 20:10,20150801 20:10,3,1,챔스 섬머 배준식 인터뷰 솔로 랭크 올리 멘탈 중요,오늘 승리 소감 어떻게 오늘 경기 오랜만 배성웅 이지훈 이지훈 오랜만 나왔 때문 으...,헝그리앱,오늘 승리 소감 어떻게 오늘 경기 오랜만 배성웅 이지훈 이지훈 오랜만 나왔 때문 으...,0.0,416
3,20150801,20150801 23:31,20150801 23:31,4,1,챔스 섬머 베누 소닉붐 꼴찌 반격 개시 동안 설움 설욕 대망 승리,챔스 섬머 경기 세트 기세 올린 베누 소닉붐 승리 가져갔 때문 진에어 그린 윙스 에...,헝그리앱,챔스 섬머 경기 세트 기세 올린 베누 소닉붐 승리 가져갔 때문 진에어 그린 윙스 에...,0.0,0
4,20150801,20150801 21:58,20150801 21:58,5,1,챔스 섬머 베누 소닉붐 대승 기반 으로 세트 승리,챔스 섬머 경기 세트 시작 세트 진에어 그린 윙스 에게 억눌린 베누 소닉붐 세트 미...,헝그리앱,챔스 섬머 경기 세트 시작 세트 진에어 그린 윙스 에게 억눌린 베누 소닉붐 세트 미...,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
51811,20200430,20200430 17:49,20200430 17:56,26,1,lck 승강 다이브 받아치 승리 샌드박스 서라벌 세트 선승,종각 lol 파크 에서 열린 열린 lck 섬머 스플릿 승강 최종 세트 에서 샌드박스...,인벤,종각 lol 파크 에서 열린 열린 lck 섬머 스플릿 승강 최종 세트 에서 샌드박스...,0.0,1166
51812,20200430,20200430 16:14,20200430 16:31,27,1,챔스 승강 서라벌 샌드박스 베스트 라인업 으로 맞대결,lck 서머 최종 에서 대결 서라벌 게이밍 샌드박스 게이밍 사진 라이엇 게임즈 제공...,데일리e스포츠,마지막 승강 에서 승리 원하 서라벌 게이밍 샌드박스 게이밍 승강 내내 기용 베스트 ...,0.0,1057
51813,20200430,20200430 17:50,20200430 17:50,28,1,일방 전투 승리 샌드박스 게이밍 서라벌 게이밍 상대 세트 승리,우리은행 lck spring split 승강 최종진 출전 에서 샌드박스 게이밍 서라...,엑스포츠뉴스,우리은행 lck spring split 승강 최종진 출전 에서 샌드박스 게이밍 서라...,0.0,835
51814,20200430,20200430 18:05,20200430 18:05,29,1,lck 승강 샌드박스 절묘 카운터 손쉽 기선 제압,osen 고용준 기자 lck 최후 승강 에서 샌드박스 먼저 다이브 공세 절묘 받아치...,OSEN,lck 최후 승강 에서 샌드박스 먼저 다이브 공세 절묘 받아치 면서 흐름 샌드박스 ...,0.0,490


In [8]:
clean_jamo_df = clean_df.copy()
for column in TEXT_COLUMNS:
    clean_jamo_df[column] = clean_jamo_df[column].progress_apply(lambda x: jamo.j2hcj(jamo.h2j(x)) if x is not np.nan else x)
clean_jamo_df.to_csv(tokenizing_dir / ('news-%s-%s-clean-jamo.csv' % (DATA_TYPE, MECAB_TYPE)), index=False)
clean_jamo_df

,date,date_input,date_modify,ranking,thumbnail,title,lede,office,contents,vod,view
0,20150801,20150801 12:49,20150801 12:49,1,1,ㅊㅐㅁㅅㅡ ㅁㅐㅊㅣ ㄹㅣㅂㅠ ㅂㅜㄹㄹㅑㅇ ㅎㅏㄱㅅㅐㅇ ㄴㅗㄷㅗㅇㅎㅕㄴ ㅅㅜㄴㅡㅇ ㄷ...,ㅌㅏㄹㅅㅓㄴ ㅇㅓㅇㅛ ㅅㅜㄴㅡㅇ ㅁㅏㄴㅈㅓㅁ koo ㅌㅏㅇㅣㄱㅓㅈㅡ kt ㄹㅗㄹㅅㅡ...,인벤,ㅌㅏㄹㅅㅓㄴ ㅇㅓㅇㅛ ㅅㅜㄴㅡㅇ ㅁㅏㄴㅈㅓㅁ koo ㅌㅏㅇㅣㄱㅓㅈㅡ kt ㄹㅗㄹㅅㅡ...,0.0,198042
1,20150801,20150801 23:32,20150801 23:32,2,1,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㅋㅡㄹㄹㅣㅇㅓ ㅅㅏㅅㅣㄴ ㅇㅣㄴㅌㅓㅂ...,ㄱㅡㅁㅇㅣㄹ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅅㅣㅈㅡㄴ ㅅㅏㅅㅏㅇ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㅅ...,헝그리앱,ㄱㅡㅁㅇㅣㄹ ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅅㅣㅈㅡㄴ ㅅㅏㅅㅏㅇ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㅅ...,0.0,8657
2,20150801,20150801 20:10,20150801 20:10,3,1,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅂㅐㅈㅜㄴㅅㅣㄱ ㅇㅣㄴㅌㅓㅂㅠ ㅅㅗㄹㄹㅗ ㄹㅐㅇㅋㅡ ㅇㅗㄹㄹㅣ...,ㅇㅗㄴㅡㄹ ㅅㅡㅇㄹㅣ ㅅㅗㄱㅏㅁ ㅇㅓㄸㅓㅎㄱㅔ ㅇㅗㄴㅡㄹ ㄱㅕㅇㄱㅣ ㅇㅗㄹㅐㄴㅁㅏㄴ...,헝그리앱,ㅇㅗㄴㅡㄹ ㅅㅡㅇㄹㅣ ㅅㅗㄱㅏㅁ ㅇㅓㄸㅓㅎㄱㅔ ㅇㅗㄴㅡㄹ ㄱㅕㅇㄱㅣ ㅇㅗㄹㅐㄴㅁㅏㄴ...,0.0,416
3,20150801,20150801 23:31,20150801 23:31,4,1,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㄲㅗㄹㅉㅣ ㅂㅏㄴㄱㅕㄱ ㄱㅐㅅㅣ ㄷㅗ...,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㄱㅕㅇㄱㅣ ㅅㅔㅌㅡ ㄱㅣㅅㅔ ㅇㅗㄹㄹㅣㄴ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂ...,헝그리앱,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㄱㅕㅇㄱㅣ ㅅㅔㅌㅡ ㄱㅣㅅㅔ ㅇㅗㄹㄹㅣㄴ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂ...,0.0,0
4,20150801,20150801 21:58,20150801 21:58,5,1,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㅂㅔㄴㅜ ㅅㅗㄴㅣㄱㅂㅜㅁ ㄷㅐㅅㅡㅇ ㄱㅣㅂㅏㄴ ㅇㅡㄹㅗ ㅅㅔㅌ...,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㄱㅕㅇㄱㅣ ㅅㅔㅌㅡ ㅅㅣㅈㅏㄱ ㅅㅔㅌㅡ ㅈㅣㄴㅇㅔㅇㅓ ㄱㅡㄹㅣ...,헝그리앱,ㅊㅐㅁㅅㅡ ㅅㅓㅁㅁㅓ ㄱㅕㅇㄱㅣ ㅅㅔㅌㅡ ㅅㅣㅈㅏㄱ ㅅㅔㅌㅡ ㅈㅣㄴㅇㅔㅇㅓ ㄱㅡㄹㅣ...,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
51811,20200430,20200430 17:49,20200430 17:56,26,1,lck ㅅㅡㅇㄱㅏㅇ ㄷㅏㅇㅣㅂㅡ ㅂㅏㄷㅇㅏㅊㅣ ㅅㅡㅇㄹㅣ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅅㅓㄹ...,ㅈㅗㅇㄱㅏㄱ lol ㅍㅏㅋㅡ ㅇㅔㅅㅓ ㅇㅕㄹㄹㅣㄴ ㅇㅕㄹㄹㅣㄴ lck ㅅㅓㅁㅁㅓ ㅅ...,인벤,ㅈㅗㅇㄱㅏㄱ lol ㅍㅏㅋㅡ ㅇㅔㅅㅓ ㅇㅕㄹㄹㅣㄴ ㅇㅕㄹㄹㅣㄴ lck ㅅㅓㅁㅁㅓ ㅅ...,0.0,1166
51812,20200430,20200430 16:14,20200430 16:31,27,1,ㅊㅐㅁㅅㅡ ㅅㅡㅇㄱㅏㅇ ㅅㅓㄹㅏㅂㅓㄹ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅂㅔㅅㅡㅌㅡ ㄹㅏㅇㅣㄴㅇㅓ...,lck ㅅㅓㅁㅓ ㅊㅚㅈㅗㅇ ㅇㅔㅅㅓ ㄷㅐㄱㅕㄹ ㅅㅓㄹㅏㅂㅓㄹ ㄱㅔㅇㅣㅁㅣㅇ ㅅㅐㄴㄷ...,데일리e스포츠,ㅁㅏㅈㅣㅁㅏㄱ ㅅㅡㅇㄱㅏㅇ ㅇㅔㅅㅓ ㅅㅡㅇㄹㅣ ㅇㅝㄴㅎㅏ ㅅㅓㄹㅏㅂㅓㄹ ㄱㅔㅇㅣㅁㅣ...,0.0,1057
51813,20200430,20200430 17:50,20200430 17:50,28,1,ㅇㅣㄹㅂㅏㅇ ㅈㅓㄴㅌㅜ ㅅㅡㅇㄹㅣ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㄱㅔㅇㅣㅁㅣㅇ ㅅㅓㄹㅏㅂㅓㄹ ...,ㅇㅜㄹㅣㅇㅡㄴㅎㅐㅇ lck spring split ㅅㅡㅇㄱㅏㅇ ㅊㅚㅈㅗㅇㅈㅣㄴ ㅊㅜ...,엑스포츠뉴스,ㅇㅜㄹㅣㅇㅡㄴㅎㅐㅇ lck spring split ㅅㅡㅇㄱㅏㅇ ㅊㅚㅈㅗㅇㅈㅣㄴ ㅊㅜ...,0.0,835
51814,20200430,20200430 18:05,20200430 18:05,29,1,lck ㅅㅡㅇㄱㅏㅇ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅈㅓㄹㅁㅛ ㅋㅏㅇㅜㄴㅌㅓ ㅅㅗㄴㅅㅟㅂ ㄱㅣㅅ...,osen ㄱㅗㅇㅛㅇㅈㅜㄴ ㄱㅣㅈㅏ lck ㅊㅚㅎㅜ ㅅㅡㅇㄱㅏㅇ ㅇㅔㅅㅓ ㅅㅐㄴㄷㅡㅂ...,OSEN,lck ㅊㅚㅎㅜ ㅅㅡㅇㄱㅏㅇ ㅇㅔㅅㅓ ㅅㅐㄴㄷㅡㅂㅏㄱㅅㅡ ㅁㅓㄴㅈㅓ ㄷㅏㅇㅣㅂㅡ ㄱ...,0.0,490
